### Agents

Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [2]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [8]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Whats up, how are you doing today?")
result.final_output

"I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

In [7]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Manchester United"
result = await Runner.run(joke_agent, topic)
result.final_output

"Why don't Manchester United players play hide and seek?\n\nBecause good luck hiding when the entire city can see you haven't scored!"

In [10]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to French: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the Manchester United fan bring a ladder to the game?

Because he heard the trophies were up for grabs!

Translated joke:
Pourquoi le supporter de Manchester United a-t-il apporté une échelle au match ?

Parce qu'il avait entendu dire que les trophées étaient à saisir !


### Structured Outputs

Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [14]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Oatmeal with Almond Butter")
recipe = response.final_output
print(recipe)

title='Creamy Almond Butter Oatmeal' ingredients=['1 cup rolled oats', '2 cups almond milk or water', '2 tablespoons almond butter', '1 tablespoon honey or maple syrup', '1 teaspoon vanilla extract', '1/2 teaspoon cinnamon', 'Pinch of salt', 'Sliced almonds (optional, for topping)', 'Fresh berries (optional, for topping)'] cooking_time=10 servings=2
